In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))

In [ ]:
virtual_display.start()

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-cgrd44gc
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-cgrd44gc
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/simoninithomas/gym-games to /tmp/pip-req-build-j6gwesm7
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /tmp/pip-req-build-j6gwesm7
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 13.0 MB/s eta 0:00:00
  Created wheel for ple: filename=ple-0.0.1-py3-none-any.whl size=50771 sha256=30abd88a0b3eae976108c96d292b6027147c97567397414d07d7a87b11fdba17
  Stored in dir

In [ ]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
import gym_pygame

In [ ]:
from huggingface_hub import notebook_login
import imageio

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
env_id = 'CartPole-v1'

env = gym.make(env_id, new_step_api=True)

eval_env = gym.make(env_id, new_step_api=True)

s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print('___Observation Space___\n')
print(f'The State Space is: {s_size}')
print(f'Space Space Sample: {env.observation_space.sample()}')

___Observation Space___

The State Space is: 4
Space Space Sample: [ 2.8948383e+00 -1.8582466e+38  1.9793138e-01 -2.0232744e+37]


In [ ]:
print('___Action Space___\n')
print(f'The Action Space is: {a_size}')
print(f'Action Space Sample: {env.action_space.sample()}')

___Action Space___

The Action Space is: 2
Action Space Sample: 0


In [ ]:
class Policy(nn.Module):
  def __init__(self, s_size, a_size, h_size):
    super(Policy, self).__init__()
    self.fc1 = nn.Linear(s_size, h_size)
    self.fc2 = nn.Linear(h_size, a_size)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.softmax(x, dim=1)

  def act(self, state):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    probs = self.forward(state).cpu()
    m = Categorical(probs=probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

(1, tensor([-0.7329], grad_fn=<SqueezeBackward1>))

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
  score_deque = deque(maxlen=100)
  scores = []

  for i_episode in range(1, n_training_episodes + 1):
    saved_log_probs = []
    rewards = []
    state = env.reset()

    for t in range(max_t):
      action, log_prob = policy.act(state)
      saved_log_probs.append(log_prob)
      state, reward, done, _, _ = env.step(action)
      rewards.append(reward)
      if done:
        break
    score_deque.append(sum(rewards))
    scores.append(sum(rewards))

    returns = deque(maxlen=max_t)
    n_steps = len(rewards)

    for t in range(n_steps)[::-1]:
      disc_return_t = returns[0] if len(returns) > 0 else 0
      returns.appendleft(gamma * disc_return_t + rewards[t])

    eps = np.finfo(np.float32).eps.item()

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    policy_loss = []
    for log_prob, disc_return in zip(saved_log_probs, returns):
      policy_loss.append(-log_prob * disc_return)
    policy_loss = torch.cat(policy_loss).sum()

    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

    if i_episode % print_every == 0:
      print(f'Episode {i_episode}\tAverage Score: {np.mean(score_deque):.2f}')
  return scores

In [ ]:
cartpole_hyperparameters = {
    'h_size': 16,
    'n_trainig_episodes': 1000,
    'n_evaluation_episodes': 10,
    'max_t': 1000,
    'gamma': 1.0,
    'lr': 1e-2,
    'env_id': env_id,
    'state_space': s_size,
    'action_space': a_size
}

In [ ]:
cartpole_policy = Policy(
    cartpole_hyperparameters['state_space'],
    cartpole_hyperparameters['action_space'],
    cartpole_hyperparameters['h_size']
).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters['lr'])

In [ ]:
scores = reinforce(
     cartpole_policy,
    cartpole_optimizer,
    cartpole_hyperparameters['n_trainig_episodes'],
    cartpole_hyperparameters['max_t'],
    cartpole_hyperparameters['gamma'],
    100
)

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, _, _ = env.step(action)
      total_rewards_ep += reward

      if done:
        break

      state = new_state

    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [ ]:
evaluate_agent(
    eval_env, cartpole_hyperparameters['max_t'], cartpole_hyperparameters['n_evaluation_episodes'], cartpole_policy
)

(911.0, 267.0)

In [ ]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

In [ ]:
def record_video(env, policy, out_directory, fps=30):
    images = []
    done = False
    state = env.reset()
    img = env.render(mode="rgb_array")
    images.append(img)
    while not done:
        action, _ = policy.act(state)
        state, reward, done, _, _ = env.step(action)
        img = env.render(mode="rgb_array")
        images.append(img)
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id,
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):

  _, repo_name = repo_id.split("/")
  api = HfApi()

  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)

    torch.save(model, local_directory / "model.pt")

    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)

    mean_reward, std_reward = evaluate_agent(eval_env,
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"],
                                            model)

    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"],
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    env_name = hyperparameters["env_id"]

    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    metadata_save(readme_path, metadata)

    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")

In [ ]:
notebook_login()

In [ ]:
repo_id = 'khadivi-ah/Reinforce-cartpole'
push_to_hub(
    repo_id,
    cartpole_policy,
    cartpole_hyperparameters,
    eval_env,
    video_fps=30
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


model.pt:   0%|          | 0.00/3.26k [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here: https://huggingface.co/khadivi-ah/Reinforce-cartpole


# Pixelcopter

In [ ]:
env_id = 'Pixelcopter-PLE-v0'
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

couldn't import doomish
Couldn't import doom


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
print('___Observation Space__\n')
print(f'The State Space is: {s_size}')
print(f'Sample observation {env.observation_space.sample()}')

___Observation Space__

The State Space is: 7
Sample observation [ 0.645371    1.1769879  -1.2804466  -0.29793003 -0.6003041   1.220186
 -1.5524642 ]


In [ ]:
print('___Action Space___\n')
print(f'The Action Space is: {a_size}')
print(f'Action Space Sample {env.action_space.sample()}')

___Action Space___

The Action Space is: 2
Action Space Sample 1


In [ ]:
class Policy(nn.Module):
  def __init__(self, s_size, a_size, h_size):
    super(Policy, self).__init__()
    self.fc1 = nn.Linear(s_size, h_size)
    self.fc2 = nn.Linear(h_size, h_size*2)
    self.fc3 = nn.Linear(h_size*2, a_size)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.softmax(x, dim=1)

  def act(self, state):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    probs = self.forward(state).cpu()
    m = Categorical(probs)
    action = m.sample()
    return action.item(), m.log_prob(action)

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
  score_deque = deque(maxlen=100)
  scores = []

  for i_episode in range(1, n_training_episodes + 1):
    saved_log_probs = []
    rewards = []
    state = env.reset()

    for t in range(max_t):
      action, log_prob = policy.act(state)
      saved_log_probs.append(log_prob)
      state, reward, done, _ = env.step(action)
      rewards.append(reward)
      if done:
        break
    score_deque.append(sum(rewards))
    scores.append(sum(rewards))

    returns = deque(maxlen=max_t)
    n_steps = len(rewards)

    for t in range(n_steps)[::-1]:
      disc_return_t = returns[0] if len(returns) > 0 else 0
      returns.appendleft(gamma * disc_return_t + rewards[t])

    eps = np.finfo(np.float32).eps.item()

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    policy_loss = []
    for log_prob, disc_return in zip(saved_log_probs, returns):
      policy_loss.append(-log_prob * disc_return)
    policy_loss = torch.cat(policy_loss).sum()

    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

    if i_episode % print_every == 0:
      print(f'Episode {i_episode}\tAverage Score: {np.mean(score_deque):.2f}')
  return scores

In [ ]:
pixelcopter_hyperparameters = {
    'h_size': 64,
    'n_training_episodes': 40000,
    'n_evaluation_episodes': 10,
    'max_t': 10000,
    'gamma': 0.99,
    'lr': 1e-4,
    'env_id': env_id,
    'state_space': s_size,
    'action_space': a_size
}

In [ ]:
torch.manual_seed(50)
pixelcopter_policy = Policy(
    pixelcopter_hyperparameters['state_space'],
    pixelcopter_hyperparameters['action_space'],
    pixelcopter_hyperparameters['h_size']
).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters['lr'])

In [ ]:
scores = reinforce(
    pixelcopter_policy,
    pixelcopter_optimizer,
    pixelcopter_hyperparameters['n_training_episodes'],
    pixelcopter_hyperparameters['max_t'],
    pixelcopter_hyperparameters['gamma'],
    1000
)

Episode 1000	Average Score: 3.51
Episode 2000	Average Score: 5.44
Episode 3000	Average Score: 8.46
Episode 4000	Average Score: 9.70
Episode 5000	Average Score: 10.68
Episode 6000	Average Score: 13.43
Episode 7000	Average Score: 14.09
Episode 8000	Average Score: 14.87
Episode 9000	Average Score: 20.35
Episode 10000	Average Score: 23.01
Episode 11000	Average Score: 17.60
Episode 12000	Average Score: 22.06
Episode 13000	Average Score: 25.06
Episode 14000	Average Score: 24.57
Episode 15000	Average Score: 23.68
Episode 16000	Average Score: 25.79
Episode 17000	Average Score: 25.87
Episode 18000	Average Score: 28.10
Episode 19000	Average Score: 33.07
Episode 20000	Average Score: 27.89
Episode 21000	Average Score: 33.56
Episode 22000	Average Score: 30.54
Episode 23000	Average Score: 35.53
Episode 24000	Average Score: 33.23
Episode 25000	Average Score: 34.85
Episode 26000	Average Score: 42.29
Episode 27000	Average Score: 40.34
Episode 28000	Average Score: 28.28
Episode 29000	Average Score: 35.3

KeyboardInterrupt: 

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, _ = env.step(action)
      total_rewards_ep += reward

      if done:
        break

      state = new_state

    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [ ]:
evaluate_agent(
    eval_env, pixelcopter_hyperparameters['max_t'], pixelcopter_hyperparameters['n_evaluation_episodes'], pixelcopter_policy
)

(29.8, 20.783647418102532)

In [ ]:
notebook_login()

In [ ]:
def record_video(env, policy, out_directory, fps=30):
    images = []
    done = False
    state = env.reset()
    img = env.render(mode="rgb_array")
    images.append(img)
    while not done:
        action, _ = policy.act(state)
        state, reward, done, _ = env.step(action)
        img = env.render(mode="rgb_array")
        images.append(img)
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
repo_id = 'khadivi-ah/Reinforce-Pixelcopter'
push_to_hub(
    repo_id,
    pixelcopter_policy,
    pixelcopter_hyperparameters,
    eval_env,
    video_fps=30
)

model.pt:   0%|          | 0.00/39.7k [00:00<?, ?B/s]

Your model is pushed to the Hub. You can view your model here: https://huggingface.co/khadivi-ah/Reinforce-Pixelcopter
